# Visualize pymoo result
Load the pymoo result object saved with pickle and visualize it.

ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

### Load rnsga3 results for all topos

In [2]:
with open('res_bc_Cogent_rnsga3.pkl','rb') as f_Cogent:
    res_6_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_rnsga3.pkl','rb') as f_UsCarrier:
    res_6_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_rnsga3.pkl','rb') as f_HiberniaGlobal:
    res_6_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_rnsga3.pkl','rb') as f_Colt:
    res_6_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_rnsga3.pkl','rb') as f_Funet:
    res_6_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_rnsga3.pkl','rb') as f_Abvt:
    res_6_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_rnsga3.pkl','rb') as f_Intellifiber:
    res_6_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_rnsga3.pkl','rb') as f_TataNld:
    res_6_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_rnsga3.pkl','rb') as f_Internode:
    res_6_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_rnsga3.pkl','rb') as f_Missouri:
    res_6_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_rnsga3.pkl','rb') as f_Ion:
    res_6_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_rnsga3.pkl','rb') as f_Palmetto:
    res_6_Palmetto = pickle.load(f_Palmetto)

## Hypervolume
1. Store values

In [ ]:
# rnsga3
F6_Cogent=res_6_Cogent.F
F6_UsCarrier=res_6_UsCarrier.F
F6_HiberniaGlobal=res_6_HiberniaGlobal.F
F6_Colt=res_6_Colt.F
F6_Funet=res_6_Funet.F
F6_Abvt=res_6_Abvt.F
F6_Intellifiber=res_6_Intellifiber.F
F6_TataNld=res_6_TataNld.F
F6_Internode=res_6_Internode.F
F6_Missouri=res_6_Missouri.F
F6_Ion=res_6_Ion.F
F6_Palmetto=res_6_Palmetto.F

# # Nadir Point from 9 Algorithms: 
# ref_point = [1.81000000e+02 6.20000000e+01 3.92281875e+03 1.70537209e+00]
# # Nadir Point from 4 Algorithms: 
# ref_point = [160.  77.  inf  inf]
# # Nadir Point from all (13) Algorithms: 
# ref_point = [181.  77.  inf  inf]

ref_point = [181, 77, 3922.81875, 1.70537209]

hist_F6_Cogent = []
hist_F6_UsCarrier = []
hist_F6_HiberniaGlobal = []
hist_F6_Colt = []
hist_F6_Funet = []
hist_F6_Abvt = []
hist_F6_Intellifiber = []
hist_F6_TataNld = []
hist_F6_Internode = []
hist_F6_Missouri = []
hist_F6_Ion = []
hist_F6_Palmetto = []


for algo6_Cogent in res_6_Cogent.history:
    opt6_Cogent = algo6_Cogent.opt
    feas6_Cogent = np.where(opt6_Cogent.get("feasible"))[0]
    hist_F6_Cogent.append(opt6_Cogent.get("F")[feas6_Cogent])
for algo6_UsCarrier in res_6_UsCarrier.history:
    opt6_UsCarrier = algo6_UsCarrier.opt
    feas6_UsCarrier = np.where(opt6_UsCarrier.get("feasible"))[0]
    hist_F6_UsCarrier.append(opt6_UsCarrier.get("F")[feas6_UsCarrier])
for algo6_HiberniaGlobal in res_6_HiberniaGlobal.history:
    opt6_HiberniaGlobal = algo6_HiberniaGlobal.opt
    feas6_HiberniaGlobal = np.where(opt6_HiberniaGlobal.get("feasible"))[0]
    hist_F6_HiberniaGlobal.append(opt6_HiberniaGlobal.get("F")[feas6_HiberniaGlobal])
for algo6_Colt in res_6_Colt.history:
    opt6_Colt = algo6_Colt.opt
    feas6_Colt = np.where(opt6_Colt.get("feasible"))[0]
    hist_F6_Colt.append(opt6_Colt.get("F")[feas6_Colt])
for algo6_Funet in res_6_Funet.history:
    opt6_Funet = algo6_Funet.opt
    feas6_Funet = np.where(opt6_Funet.get("feasible"))[0]
    hist_F6_Funet.append(opt6_Funet.get("F")[feas6_Funet])
for algo6_Abvt in res_6_Abvt.history:
    opt6_Abvt = algo6_Abvt.opt
    feas6_Abvt = np.where(opt6_Abvt.get("feasible"))[0]
    hist_F6_Abvt.append(opt6_Abvt.get("F")[feas6_Abvt])
for algo6_Intellifiber in res_6_Intellifiber.history:
    opt6_Intellifiber = algo6_Intellifiber.opt
    feas6_Intellifiber = np.where(opt6_Intellifiber.get("feasible"))[0]
    hist_F6_Intellifiber.append(opt6_Intellifiber.get("F")[feas6_Intellifiber])
for algo6_TataNld in res_6_TataNld.history:
    opt6_TataNld = algo6_TataNld.opt
    feas6_TataNld = np.where(opt6_TataNld.get("feasible"))[0]
    hist_F6_TataNld.append(opt6_TataNld.get("F")[feas6_TataNld])
for algo6_Internode in res_6_Internode.history:
    opt6_Internode = algo6_Internode.opt
    feas6_Internode= np.where(opt6_Internode.get("feasible"))[0]
    hist_F6_Internode.append(opt6_Internode.get("F")[feas6_Internode])
for algo6_Missouri in res_6_Missouri.history:
    opt6_Missouri = algo6_Missouri.opt
    feas6_Missouri = np.where(opt6_Missouri.get("feasible"))[0]
    hist_F6_Missouri.append(opt6_Missouri.get("F")[feas6_Missouri])
for algo6_Ion in res_6_Ion.history:
    opt6_Ion = algo6_Ion.opt
    feas6_Ion = np.where(opt6_Ion.get("feasible"))[0]
    hist_F6_Ion.append(opt6_Ion.get("F")[feas6_Ion])
for algo6_Palmetto in res_6_Palmetto.history:
    opt6_Palmetto = algo6_Palmetto.opt
    feas6_Palmetto = np.where(opt6_Palmetto.get("feasible"))[0]
    hist_F6_Palmetto.append(opt6_Palmetto.get("F")[feas6_Palmetto])

2. Calculate hypervolume and merge them (Mean value) for rnsga3 algorithm

In [ ]:
from pymoo.indicators.hv import Hypervolume

metric = Hypervolume(ref_point= np.array(ref_point))

hv_6_Cogent = [metric.do(_F6_Cogent) for _F6_Cogent in hist_F6_Cogent]
hv_6_UsCarrier = [metric.do(_F6_UsCarrier) for _F6_UsCarrier in hist_F6_UsCarrier]
hv_6_HiberniaGlobal = [metric.do(_F6_HiberniaGlobal) for _F6_HiberniaGlobal in hist_F6_HiberniaGlobal]
hv_6_Colt = [metric.do(_F6_Colt) for _F6_Colt in hist_F6_Colt]
hv_6_Funet = [metric.do(_F6_Funet) for _F6_Funet in hist_F6_Funet]
hv_6_Abvt = [metric.do(_F6_Abvt) for _F6_Abvt in hist_F6_Abvt]
hv_6_Intellifiber = [metric.do(_F6_Intellifiber) for _F6_Intellifiber in hist_F6_Intellifiber]
hv_6_TataNld = [metric.do(_F6_TataNld) for _F6_TataNld in hist_F6_TataNld]
hv_6_Internode = [metric.do(_F6_Internode) for _F6_Internode in hist_F6_Internode]
hv_6_Missouri = [metric.do(_F6_Missouri) for _F6_Missouri in hist_F6_Missouri]
hv_6_Ion = [metric.do(_F6_Ion) for _F6_Ion in hist_F6_Ion]
hv_6_Palmetto = [metric.do(_F6_Palmetto) for _F6_Palmetto in hist_F6_Palmetto]

# rnsga3
hv_6 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_6_Cogent, hv_6_UsCarrier, hv_6_HiberniaGlobal, hv_6_Colt, hv_6_Funet, hv_6_Abvt, hv_6_Intellifiber, hv_6_TataNld, hv_6_Internode, hv_6_Missouri, hv_6_Ion, hv_6_Palmetto)]

In [ ]:
print('The hypervolumn for Cogent:')
print(hv_6_Cogent)
print('The hypervolumn for UsCarrier:')
print(hv_6_UsCarrier)
print('The hypervolumn for HiberniaGlobal:')
print(hv_6_HiberniaGlobal)
print('The hypervolumn for Colt:')
print(hv_6_Colt)
print('The hypervolumn for Funet:')
print(hv_6_Funet)
print('The hypervolumn for Abvt:')
print(hv_6_Abvt)
print('The hypervolumn for Intellifiber:')
print(hv_6_Intellifiber)
print('The hypervolumn for TataNld:')
print(hv_6_TataNld)
print('The hypervolumn for Internode:')
print(hv_6_Internode)
print('The hypervolumn for Missouri:')
print(hv_6_Missouri)
print('The hypervolumn for Ion:')
print(hv_6_Ion)
print('The hypervolumn for Palmetto:')
print(hv_6_Palmetto)

# Show all hv for each topo
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_6_Cogent)+1)), hv_6_Cogent,  color='black', label='Cogent')
plt.plot(list(range(1, len(hv_6_UsCarrier)+1)), hv_6_UsCarrier,  color='blue', label='UsCarrier')
plt.plot(list(range(1, len(hv_6_HiberniaGlobal)+1)), hv_6_HiberniaGlobal,  color='red', label='HiberniaGlobal')
plt.plot(list(range(1, len(hv_6_Colt)+1)), hv_6_Colt,  color='orange', label='Colt')
plt.plot(list(range(1, len(hv_6_Funet)+1)), hv_6_Funet,  color=(0, 0, 1, 0.5), label='Funet')
plt.plot(list(range(1, len(hv_6_Abvt)+1)), hv_6_Abvt,  color='gray', label='Abvt')
plt.plot(list(range(1, len(hv_6_Intellifiber)+1)), hv_6_Intellifiber,  color='purple', label='Intellifiber')
plt.plot(list(range(1, len(hv_6_TataNld)+1)), hv_6_TataNld,  color='green', label='TataNld')
plt.plot(list(range(1, len(hv_6_Internode)+1)), hv_6_Internode,  color='#33FF57', label='Internode')
plt.plot(list(range(1, len(hv_6_Missouri)+1)), hv_6_Missouri,  color='brown', label='Missouri')
plt.plot(list(range(1, len(hv_6_Ion)+1)), hv_6_Ion,  color='pink', label='Ion')
plt.plot(list(range(1, len(hv_6_Palmetto)+1)), hv_6_Palmetto,  color='olive', label='Palmetto')
plt.title("Convergence")
plt.xlabel("Topos")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

In [ ]:
print('The aveage of hypervolumn for rnsga3:')
print(hv_6)

3. Draw mean hypervolume for rnsga3

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations") # brown,pink,#5733FF,olive,cyan,#FF5733,#33FF57
plt.ylabel("Hypervolume")
plt.legend()
plt.show()